In [2]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient()

In [3]:
#System-specific setup

db = client.forsaken
book_schemas = db.rawSchemas

In [4]:
def get_keys(collection):
    KEY_LIST_NAME = "allkeys"
    
    key_aggregation = [
      {"$project":{"arrayofkeyvalue":{"$objectToArray":"$$ROOT"}}},
      {"$unwind":"$arrayofkeyvalue"},
      {"$group":{"_id":None ,KEY_LIST_NAME:{"$addToSet":"$arrayofkeyvalue.k"}}}
    ]
    
    aggregation_cursor = collection.aggregate(key_aggregation)
    aggregation_dict = aggregation_cursor.next()
    key_list = aggregation_dict[KEY_LIST_NAME]
    
    return key_list

In [47]:
def find_all_values(key, collection):
    return collection.distinct(key)

In [48]:
def collection_len(collection):
    return collection.count_documents(dict())

In [61]:
def documents_generator(collection, num_of_documents):
    offset = 0
    amount_of_documents = collection_len(collection)
    
    while offset < amount_of_documents:
        yield list(collection.find().skip(offset).limit(num_of_documents))
        
        offset += num_of_documents